In [1]:
using RandomMeas

We first define the system size (number of qubits) we are going to be interested in.

In [2]:
N=20

20

### Generation of randomized measurment settings

Next, we sample N_U = 100 measurement settings. Here, we consider local (single-qubit) random unitaries which are sampled, for each qubit independently, sample from the Haar measure on U(2).

In [3]:
NU = 100 # Number of measurement settings
measurement_settings = [LocalUnitaryMeasurementSetting(N) for _ in 1:NU];

### Classical simulation of randomized measurements

We could now export these measurement settings and perform randomized measurements on a QPU. Here, instead, we simulate the randomized measurements on a CPU on a sample MPS \psi.

We begin by defining the corresponding site index set and generate a weakly entangled random MPS (bond dimension 2). 

In [4]:
site_indices = siteinds("Qubit",N)
ψ = random_mps(site_indices, linkdims=2); # state

We now sample N_M = 100 projective randomized measurements per measurement setting.

In [6]:
NM=100 # Number of projective measurements per measurement setting
measurement_data = MeasurementGroup(ψ ,measurement_settings,NM);

### Post-processing

Now, we start the postprocessing. We first construct classical shadows (reference). Here, "factorized" classical shadows are memory efficient objects (stored as N x 2 x 2 arrays) suitable for large system sizes.

In [14]:
classical_shadows = get_factorized_shadows(measurement_data);

Next, we use the classical shadows to estimat the expectation value of a simple MPO O.

In [9]:
ops = ["I" for _ in 1:N]
ops[1] = "Z"
ops[4] = "X"
O=MPO(site_indices,ops);

mean_val, sem_val = real.(get_expect_shadow(O,classical_shadows,compute_sem = true));
exact_val = inner(ψ',O,ψ);

println("Estimated expectation value ⟨O⟩ = $(round(mean_val, digits=2)) ± $(round(sem_val, digits=2))")
println("Exact expectation value of ⟨O⟩ = $(round(exact_val, digits=2))")


Estimated expectation value ⟨O⟩ = 0.07 ± 0.03
Exact expectation value of ⟨O⟩ = 0.06


Alternatively, since our MPO O has only support on qubit 1 and 4, we can first reduce the data set to effective 2-qubit system of interest. This allows to generate dense classical shadows (in 2^2 x 2^2 dense matrices) which allow for very fast post-processing (but are not memory-efficient).

In [10]:
subsystem = [1,4]
reduced_data = reduce_to_subsystem(measurement_data,subsystem);
reduced_classical_shadows = get_dense_shadows(reduced_data);

We use the reduced classical shadows to estimat the expectation value of the reduced operator O. We obtain the same value as before.

In [11]:
ops = ["Z","X"]
reduced_O=MPO([site_indices[1],site_indices[4]],ops);

mean_val, sem_val = real.(get_expect_shadow(reduced_O,reduced_classical_shadows,compute_sem = true));
exact_val = inner(reduced_O,reduce_to_subsystem(ψ,[1,4]));

println("Estimated expectation value ⟨O⟩ = $(round(mean_val, digits=2)) ± $(round(sem_val, digits=2))")
println("Exact expectation value of ⟨O⟩ = $(round(exact_val, digits=2))")


Estimated expectation value ⟨O⟩ = 0.07 ± 0.04
Exact expectation value of ⟨O⟩ = 0.06


Lastly, we can also estimate non-linear functions of the density matrices such as the purity of the reduced density matrix of subsystem defined by qubits 1 and 4. For this, we also use the reduced measurement data set.

In [ ]:
mean_val, sem_val = get_trace_moment(reduced_classical_shadows,2,compute_sem = true);
exact_val = get_trace_moment(ρ,2);

println("Estimated purity of ρ = $(round(mean_val, digits=2)) ± $(round(sem_val, digits=2))")
println("Exact purity of ρ = $(round(exact_val, digits=2))")